# RNN Text Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["this movie is good","i like this film","this movie is bad","i hate this film"]
labels = [1,1,0,0]

vocab = {}
for s in sentences:
    for w in s.split():
        if w not in vocab:
            vocab[w]=len(vocab)+1

def encode(s): return [vocab[w] for w in s.split()]
X = torch.tensor([encode(s) for s in sentences],dtype=torch.long)
y = torch.tensor(labels)

class TextRNN(nn.Module):
    def __init__(self,vocab_size,embed,hidden,out):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size,embed)
        self.rnn=nn.RNN(embed,hidden,batch_first=True)
        self.fc=nn.Linear(hidden,out)
    def forward(self,x):
        x=self.embedding(x)
        o,_=self.rnn(x)
        return self.fc(o[:,-1,:])

model=TextRNN(len(vocab)+1,10,16,2)
crit=nn.CrossEntropyLoss()
opt=optim.Adam(model.parameters(),lr=0.01)

for e in range(100):
    opt.zero_grad()
    out=model(X)
    loss=crit(out,y)
    loss.backward()
    opt.step()
    if (e+1)%20==0: print(e+1,loss.item())

def predict(s):
    t=torch.tensor([encode(s)])
    p=torch.argmax(model(t),1).item()
    return "Positive" if p==1 else "Negative"

pxrint(predict("this movie is good"),predict("this movie is bad"))

20 0.007613763213157654
40 0.0010025377850979567
60 0.0006281398818828166
80 0.0005174041725695133
100 0.0004466903628781438
Positive Negative
